# Creating a SQL database
This notebook assumes that you have PSQL already installed on your local machine and will create a PSQL database named ```drug_consumption```.

In [1]:
import pandas as pd

# Filenames

In [2]:
features_name = 'features'
features_csv_filename = "drugC_features.csv"

In [3]:
responses_name = "drug_data"
responses_csv_filename = "drugC_responses.csv"

# Split data into tables and save as .csv
Read cleaned .pkl data into pandas and split data into one dataframe per table.

In [4]:
features = (['age','gender','education','country','ethnicity','nscore',
            'escore','oscore','ascore','cscore','impulsiveness','ss'])

In [5]:
drug_list = (['alcohol','amphet','amyl','benzos','caff','cannabis','choc','coke','crack',
              'ecstasy','heroin','ketamine','legalh','lsd','meth','mushrooms','nicotine','semer','vsa'])

In [6]:
df = pd.read_pickle('./data/drug_consumption_cleaned.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1885 entries, 1 to 1888
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1885 non-null   float64
 1   gender         1885 non-null   float64
 2   education      1885 non-null   float64
 3   country        1885 non-null   float64
 4   ethnicity      1885 non-null   float64
 5   nscore         1885 non-null   float64
 6   escore         1885 non-null   float64
 7   oscore         1885 non-null   float64
 8   ascore         1885 non-null   float64
 9   cscore         1885 non-null   float64
 10  impulsiveness  1885 non-null   float64
 11  ss             1885 non-null   float64
 12  alcohol        1885 non-null   int16  
 13  amphet         1885 non-null   int16  
 14  amyl           1885 non-null   int16  
 15  benzos         1885 non-null   int16  
 16  caff           1885 non-null   int16  
 17  cannabis       1885 non-null   int16  
 18  choc    

## Features: df_features

In [7]:
df_features = df[features]
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1885 entries, 1 to 1888
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1885 non-null   float64
 1   gender         1885 non-null   float64
 2   education      1885 non-null   float64
 3   country        1885 non-null   float64
 4   ethnicity      1885 non-null   float64
 5   nscore         1885 non-null   float64
 6   escore         1885 non-null   float64
 7   oscore         1885 non-null   float64
 8   ascore         1885 non-null   float64
 9   cscore         1885 non-null   float64
 10  impulsiveness  1885 non-null   float64
 11  ss             1885 non-null   float64
dtypes: float64(12)
memory usage: 191.4 KB


## Responses (drugs): df_responses

In [8]:
df_responses = df[drug_list]
df_responses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1885 entries, 1 to 1888
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   alcohol    1885 non-null   int16
 1   amphet     1885 non-null   int16
 2   amyl       1885 non-null   int16
 3   benzos     1885 non-null   int16
 4   caff       1885 non-null   int16
 5   cannabis   1885 non-null   int16
 6   choc       1885 non-null   int16
 7   coke       1885 non-null   int16
 8   crack      1885 non-null   int16
 9   ecstasy    1885 non-null   int16
 10  heroin     1885 non-null   int16
 11  ketamine   1885 non-null   int16
 12  legalh     1885 non-null   int16
 13  lsd        1885 non-null   int16
 14  meth       1885 non-null   int16
 15  mushrooms  1885 non-null   int16
 16  nicotine   1885 non-null   int16
 17  semer      1885 non-null   int16
 18  vsa        1885 non-null   int16
dtypes: int16(19)
memory usage: 84.7 KB


## Save split data to .csv

In [9]:
df_features.to_csv('./data/'+features_csv_filename)

In [10]:
df_responses.to_csv('./data/'+responses_csv_filename)

# Database setup
Here we'll create a sql setup file that we can pass to psql in order to automate the whole process.

## Create table statement function

In [11]:
def create_table_statement(df, table_name):
    """
        Helper function to make "CREATE TABLE" statement for psql
        
        :param df: a pandas DateFrame object
        :param table_name: name of new table in psql as string
        :return create_table: "CREATE TABLE" statement for psql as string
    """
    
    col_names = df.dtypes.index.values
    dtypes = df.dtypes.values

    #every name in psql is lowercase by default
    table_name = table_name.lower()

    #statement for creating the table in psql
    create_table = "CREATE TABLE IF NOT EXISTS %s (\n" %table_name
    
    #add index
    create_table += "\tid int NOT NULL,\n"

    for idx, col_name in enumerate(col_names):
        #get the datatype for the current column
        dtype = dtypes[idx]

        #lowercase and remove spaces from column names
        col_name = col_name.lower().replace(' ', '_')
        #simply formatting so it prints more cleanly
        create_table += "\t"

        #assign column types and default values of null
        if dtype == 'int64':
            create_table = create_table + col_name + " INT DEFAULT NULL"

        elif dtype == 'int16':
            create_table = create_table + col_name + " INT DEFAULT NULL"

        elif dtype == 'object':
            create_table = create_table + col_name + " VARCHAR DEFAULT NULL"

        elif dtype == 'float64':
            create_table = create_table + col_name + " DECIMAL DEFAULT NULL"

        #add a comma and start next line on 
        #another line for cleaner print format
        #if this is not the last line
        if idx != len(col_names) - 1:
            create_table += ", \n"
    # primary key
    create_table += ",\n\tPRIMARY KEY (id)"
    #newline for cleaner print format and ; for execution
    create_table += "\n);"

    print(create_table)
    
    return create_table

## Implement table statements

In [12]:
table_features = create_table_statement(df_features, features_name)

CREATE TABLE IF NOT EXISTS features (
	id int NOT NULL,
	age DECIMAL DEFAULT NULL, 
	gender DECIMAL DEFAULT NULL, 
	education DECIMAL DEFAULT NULL, 
	country DECIMAL DEFAULT NULL, 
	ethnicity DECIMAL DEFAULT NULL, 
	nscore DECIMAL DEFAULT NULL, 
	escore DECIMAL DEFAULT NULL, 
	oscore DECIMAL DEFAULT NULL, 
	ascore DECIMAL DEFAULT NULL, 
	cscore DECIMAL DEFAULT NULL, 
	impulsiveness DECIMAL DEFAULT NULL, 
	ss DECIMAL DEFAULT NULL,
	PRIMARY KEY (id)
);


In [13]:
table_responses = create_table_statement(df_responses, responses_name )

CREATE TABLE IF NOT EXISTS drug_data (
	id int NOT NULL,
	alcohol INT DEFAULT NULL, 
	amphet INT DEFAULT NULL, 
	amyl INT DEFAULT NULL, 
	benzos INT DEFAULT NULL, 
	caff INT DEFAULT NULL, 
	cannabis INT DEFAULT NULL, 
	choc INT DEFAULT NULL, 
	coke INT DEFAULT NULL, 
	crack INT DEFAULT NULL, 
	ecstasy INT DEFAULT NULL, 
	heroin INT DEFAULT NULL, 
	ketamine INT DEFAULT NULL, 
	legalh INT DEFAULT NULL, 
	lsd INT DEFAULT NULL, 
	meth INT DEFAULT NULL, 
	mushrooms INT DEFAULT NULL, 
	nicotine INT DEFAULT NULL, 
	semer INT DEFAULT NULL, 
	vsa INT DEFAULT NULL,
	PRIMARY KEY (id)
);


## Create copy statement function

In [14]:
def create_copy_statement(table_name, table_csv_filename):
    copy_statement = "\copy "+ table_name +" FROM '"+ table_csv_filename +"' DELIMITER ',' CSV HEADER;"
    return copy_statement

## Implement copy statements

In [15]:
copy_features = create_copy_statement(features_name, './data/'+features_csv_filename)
copy_features

"\\copy features FROM './data/drugC_features.csv' DELIMITER ',' CSV HEADER;"

In [16]:
copy_responses = create_copy_statement(responses_name, './data/'+responses_csv_filename)
copy_responses

"\\copy drug_data FROM './data/drugC_responses.csv' DELIMITER ',' CSV HEADER;"

## Save statements to .sql files for use with psql

In [17]:
# table_features, table_responses, copy_features, copy_responses
sql_setup_filename = "drug_db_setup.sql"

In [18]:
chars_written = []

sql_setup_file = open(sql_setup_filename,'w')

# write() returns # of characters written - let's save that in chars_written
chars_written.append(sql_setup_file.write('\connect drug_consumption;\n\n'))

chars_written.append(sql_setup_file.write(table_features+'\n\n'))

chars_written.append(sql_setup_file.write(table_responses+'\n\n'))

chars_written.append(sql_setup_file.write(copy_features+'\n\n'))

chars_written.append(sql_setup_file.write(copy_responses+'\n\n'))

sql_setup_file.close()

print(chars_written)

[28, 456, 584, 75, 77]


# Terminal commands

Create the database.

In [19]:
# Uncomment if you haven't yet created the database
!psql -c 'CREATE DATABASE drug_consumption'

In [20]:
!echo $sql_setup_filename
!psql -f $sql_setup_filename

drug_db_setup.sql
You are now connected to database "drug_consumption" as user "kibbles".
psql:drug_db_setup.sql:18: NOTICE:  relation "features" already exists, skipping
CREATE TABLE
psql:drug_db_setup.sql:42: NOTICE:  relation "drug_data" already exists, skipping
CREATE TABLE
COPY 1885
COPY 1885


# Cleanup
Removing local temp files.

In [21]:
cleanup = True
if cleanup:
    import os
    os.remove(sql_setup_filename)
    print(sql_setup_filename, " Removed setup file!")
    os.remove('./data/'+features_csv_filename)
    os.remove('./data/'+responses_csv_filename)
    print('Removed temporary csv files in ./data/')

drug_db_setup.sql  Removed setup file!
Removed temporary csv files in ./data/
